In [5]:
"""
Compute rolling ktau values from EWS dataframe 
Edited from published codes by Bury et al. (2021), Deep learning for early warning signals of tipping points, PNAS. 
""" 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import scipy.stats as stats


# Set frequency of kendall tau measurements
ml_spacing = 10 # spacing between ML predictions
t_res = ml_spacing 

#Change your test model
#test_models are 'SIRwhiteN', 'SIRenvN', 'SIRdemN', 'SEIR', 'COVID'
test_model = 'SIRwhiteN'

# Import EWS from forced trajectories
df_ews_forced = pd.read_csv('../data/ews/df_ews_forced_{}.csv'.format(test_model))
df_ews_forced.set_index(['tsid','Time'], inplace=True)

# Import EWS from null trajectories 
df_ews_null = pd.read_csv('../data/ews/df_ews_null_{}.csv'.format(test_model))
df_ews_null.set_index(['tsid','Time'], inplace=True)

# Function to compute kendall tau for time seires data up to point t_fin
def ktau_compute(series,t_fin):
    # selected data in series where from point where measured variable
    # is defined, up to t_fin
    t_start=series[pd.notnull(series)].index[1]
    
    series_reduced = series.loc[t_start:t_fin]
    x1 = series_reduced.index.values
    x2 = series_reduced.values
    ktau, pval = stats.kendalltau(x1,x2)
    return ktau

# Compute kendall tau values at equally spaced time points
def ktau_series(series):
    tVals = series.index[::t_res]
    ktauVals = []
    for t in tVals:
        ktau = ktau_compute(series,t)
        ktauVals.append(ktau)
    
    # Return series
    ktauSeries = pd.Series(ktauVals,index=tVals)
    return ktauSeries




#-------------
# Compute kendall tau for forced simulations in I
#------------

# Store list of dfs with kendall tau values from each simulation
list_df = []

# Get EWS in x
df_ews_forced = df_ews_forced[df_ews_forced['Variable']=='I']

# Loop through each time series ID
tsid_vals = df_ews_forced.index.unique(level='tsid')

for tsid in tsid_vals:

    series_var = df_ews_forced.loc[tsid]['variance']
    series_ac = df_ews_forced.loc[tsid]['ac1']
    
    # Compute kendall tau series
    series_ktau_var = ktau_series(series_var)
    series_ktau_var.name = 'ktau_variance'
    series_ktau_ac = ktau_series(series_ac)
    series_ktau_ac.name = 'ktau_ac'

    
    # Put into temporary dataframe
    df_temp = pd.concat([series_ktau_var, series_ktau_ac], axis=1).reset_index()
    df_temp['tsid'] = tsid
    list_df.append(df_temp)
    print('Ktau computed for tsid {}'.format(tsid))
        

# Concatenate kendall tau dataframes
df_ktau_forced = pd.concat(list_df).set_index(['tsid','Time'])

# Export dataframe
df_ktau_forced.to_csv('../data/ews/df_ktau_forced_{}.csv'.format(test_model))



#-------------
# Compute kendall tau for null simulations
#------------

# Store list of dfs with kendall tau values from each simulation
list_df = []

# Get EWS in x
df_ews_null = df_ews_null[df_ews_null['Variable']=='I']

# Loop through each time series ID
tsid_vals = df_ews_null.index.unique(level='tsid')

for tsid in tsid_vals:

    series_var = df_ews_null.loc[tsid]['variance']
    series_ac = df_ews_null.loc[tsid]['ac1']
    
    # Compute kendall tau series
    series_ktau_var = ktau_series(series_var)
    series_ktau_var.name = 'ktau_variance'
    series_ktau_ac = ktau_series(series_ac)
    series_ktau_ac.name = 'ktau_ac'

    
    # Put into temporary dataframe
    df_temp = pd.concat([series_ktau_var, series_ktau_ac], axis=1).reset_index()
    df_temp['tsid'] = tsid
    list_df.append(df_temp)
    print('Ktau computed for tsid {}'.format(tsid))
        

# Concatenate kendall tau dataframes
df_ktau_null = pd.concat(list_df).set_index(['tsid','Time'])

# Export dataframe
df_ktau_null.to_csv('../data/ews/df_ktau_null_{}.csv'.format(test_model))




Ktau computed for tsid 1
Ktau computed for tsid 2
Ktau computed for tsid 3
Ktau computed for tsid 4
Ktau computed for tsid 5
Ktau computed for tsid 6
Ktau computed for tsid 7
Ktau computed for tsid 1
Ktau computed for tsid 2
Ktau computed for tsid 3
Ktau computed for tsid 4
Ktau computed for tsid 5
Ktau computed for tsid 6
Ktau computed for tsid 7
